# Agent-Based Modeling

In this set of lectures, we'll study how to design agent-based models in Python. 

> An *agent-based model* (ABM) is a simulation model in which many individual entities (*agents*) interact with each other according to fixed rules. 

ABMs are often used for modeling a wide range of social and biological systems. In fact, you've already seen an example of an ABM: the SIR model of disease spred that we studied in the previous lecture is one. There, we relied on tools from NetworkX and various other familiar programming paradigms. We'll now explore the topic of agent-based modeling from a somewhat more systematic and flexible perspective. 

There exist a large number of dedicated software packages for agent-based modeling. In this course, we'll use a relatively recent package, called [Mesa](https://mesa.readthedocs.io/en/master/index.html), for agent-based modeling in Python. To install the software, run the following code in your terminal: 

```
conda activate PIC16B
conda install -c conda-forge mesa
```

# The Schelling Model of Racial Segregation

In this set of lecture notes, we will implement the Schelling model of racial residential segregation. The Schelling model is a parable of how only *mild* individual biases can lead to highly segregated outcomes. 


In the Schelling Model, individuals of two types begin arranged randomly on a grid, which is often taken to represent a city. Not all grid squares are occupied. Here's an example starting configuration

<figure class="image" style="width:50%">
  <img src="https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/_images/schelling-screencap.png" alt="">
  <figcaption><i>An example starting configuration in the Schelling model. Image credit: <a href = "https://ncase.me/polygons/"> Vi Hart and Nicky Case</a>.</i></figcaption>
</figure>

Here's how the model works: 

1. At each timestep, agents look at their surroundings. An agent is **unhappy** if fewer than 1/3 of their neighbors have the same type, and is **happy** otherwise. 
2. All **unhappy** agents pick a random empty spot and move there. All **happy** agents stay where they are. 

We run the model until all agents are happy. The fundamental result of the model is that, even though agents have only mild biases -- they simply prefer not to be outnumbered --  acting on their preferences can still lead to highly segregated outcomes, like this: 

<figure class="image" style="width:50%">
  <img src="https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/_images/schelling-final.png" alt="">
  <figcaption><i>An example final configuration in the Schelling model. Image credit: <a href = "https://ncase.me/polygons/"> Vi Hart and Nicky Case</a>.</i></figcaption>
</figure>

For an excellent interactive demonstration of the Schelling Model, check out [this blog post](https://ncase.me/polygons/) by Vi Hart and Nicky Case.  

### A Note on History

The Schelling model does not include any concepts of historical oppression, wealth, or power, all of which contribute to racial segregation. The message of the Schelling model is that these factors are not **needed** for segregation -- mildly racist individual preferences would be enough. It is important, however, not to confuse this mathematical parable with the actual historical circumstances of racial segregation in the US or elsewhere. 

### Sources

These lecture notes are closely based on the [Schelling model example](https://github.com/projectmesa/mesa/tree/main/examples/schelling) in the [official Mesa repository](https://github.com/projectmesa/mesa). They also draw on the [Introductory Tutorial](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html) from the official Mesa documentation.  

# Implementing the Schelling Model

Let's start by implementing a bare-bones model. While there is some flexibility in how one does this, there are a few common features of most Mesa models: 

1. There must be an *agent* class, which should inherit from `mesa.Agent`. This class specifies the properties and behaviors of an individual agent in the simulation. 
    - This class must call  `mesa.Agent.__init__()` as part of its `__init__()` method. 
    - This class must have a `step()` method which describes the primary individual behavior. 
2. There must be a *model* class, which should inherit from `mesa.Model`. 
    - The `__init__()` method of this class is responsible for creating agents with their properties, as well as the space (often a grid) on which the simulation unfolds. 
    - This class must also have a `step()` method which provides a complete description of what happens in a single model time step. Often, this involves using a `Schedule` to call the `step()` method of each of the agents in some specified sequence. 
    
Let's write a very simple model that demonstrates some of these requirements. Our model won't really do very much yet, but it will demonstrate the key techniques of defining the agent and model, adding agents to the model, and calling the `step()` methods. 

Let's demonstrate the behavior of our toy model: 

Observe that, each time we call `TM.step()`, the model sweeps through the various agents and calls their individual `step()` methods. This is because we created a `RandomActivation` schedule, and added each of the agents to this schedule.

With our architecture in place, our next step is learn how to implement more interesting behaviors. 

## Spatial Grids

The Schelling model usually evolves on a grid. At the moment, we don't have a grid incorporated. Fortunately, this is easy to bring in. We simply need to add a `SingleGrid` object with specified width and height. The `torus` argument of the grid determines whether the edges "wrap around." If it is selected, then walking off the left side of the grid will put you back on the right side. This is often visualized as allowing the grid to lie on the surface of a torus, or donut: 

<figure class="image" style="width:30%">
  <img src="https://i.stack.imgur.com/ZZrv4.png" alt="">
  <figcaption><i>A toroidal grid.</i></figcaption>
</figure>

The modifications we need to make to our previous code are relatively simple: 

1. We need to give each `ToyAgent` a `pos`ition. 
2. We need to give the model a `grid` instance variable. 
3. We need to modify our initialization of agents so that we call `self.grid.position_agent(agent, pos)` in order to place each agent on the grid. 

Now we can again instantiate our model. This time, we need to pass `width`, `height`, and `density`. Here, we're creating a 10x10 grid in which roughly 10% of cells have agents in them. 

In [ ]:
TM = ToyModel(10, 10, 0.1)
TM.step()

It's also possible to directly extract the grid and visualize it using familiar tools. In a later lecture, however, we'll see some much better ways to visualize the grid. 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.imshow(np.array(TM.grid.grid) == None)

# A Basic Schelling Model

We're now ready to construct a simple version of the Schelling model. Here are the remaining ingredients we need to bring in: 

1. Agents need to have *types* associated with them. 
2. The agents `step()` method should check whether the agent is "happy" (i.e. not surrounded by too many neighbors of different `type`, and move them to an empty grid cell if not. The `SingleGrid` class we've used to create the grid provides several useful methods for handling this logic.  

Here's a function to plot the model state. Dark purple squares are empty; green squares are agents of type `triangle`, and yellow squares are agents of type `square`. 

In [ ]:
def viz_state(SM, ax):
    G = np.array(SM.grid.grid)
    to_viz = np.zeros(G.shape)
    
    for i in range(G.shape[0]):
        for j in range(G.shape[1]):
            if G[i,j] is not None:
                if G[i,j].type == "triangle":
                    to_viz[i,j] = 1.0
                elif G[i,j].type == "square":
                    to_viz[i,j] = 1.0
    ax.imshow(to_viz)

Now we're ready to visualize the evolution of our model. 

In [ ]:
fig, axarr = plt.subplots(2, 5, figsize = (10, 4))
SM = SchellingModel(20, 20, 0.9, 0.5)

t = 0
for ax in axarr.flatten():
    ax.axis("off")
    ax.set(title = f"timestep {t}")
    viz_state(SM, ax)
    t += 1
    SM.step()

We observe the characteristic separation of an initially spatially mixed population into large regions of homogeneous types. 

In coming lectures, we'll learn how to visualize these processes more gracefully; how to collect data from simulations; and how to implement more complex models. 